In [2]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
DATASET_PATH = "/content/gdrive/MyDrive/APS360_Team_15/Data_Processing/"

Mounted at /content/gdrive


In [4]:
import torch
bird_image_dataset = torch.load(os.path.join(DATASET_PATH, "bird_dataset.pt"), weights_only=False)


In [7]:
def extract_features_and_labels(image_tensor):
    image_np = image_tensor.cpu().numpy()
    if image_np.shape[0] == 3:  # If it's RGB (C, H, W), convert to grayscale
        image_np = np.transpose(image_np, (1, 2, 0))  # Convert to (H, W, C)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    else:
        image_np = np.squeeze(image_np)  # If already grayscale, remove extra dimensions

    # Extract HOG features
    hog_features = hog(image_np, pixels_per_cell=(16, 16), cells_per_block=(2, 2),
                       orientations=9, block_norm='L2-Hys', visualize=False)

    return hog_features

In [10]:
images = bird_image_dataset['images']
labels = np.array(bird_image_dataset['labels'])

# Extract HOG features for all images
hog_features_list = [extract_features_and_labels(img) for img in images]
hog_features_array = np.array(hog_features_list)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(hog_features_array, labels, test_size=0.2, random_state=42)

print("X_train length:", len(X_train))
print("y_train length:", len(y_train))
print("X_test length:", len(X_test))
print("y_test length:", len(y_test))


X_train length: 9860
y_train length: 9860
X_test length: 2465
y_test length: 2465


In [13]:
# Train SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [14]:
# Evaluate the model
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.07


In [16]:
# Save the trained model
import joblib
model_path = "/content/gdrive/MyDrive/APS360_Team_15/Models/Baseline_Models/"
joblib.dump(svm_classifier, os.path.join(model_path, "svm_bird_classifier.pkl"))

['/content/gdrive/MyDrive/APS360_Team_15/Models/Baseline_Models/svm_bird_classifier.pkl']